In [ ]:
# Write fake accuracies
import tensorflow as tf

writer = tf.summary.FileWriter('./graphs')

accuracy = [0.1, 0.4, 0.6, 0.8, 0.9, 0.95] # fake accuracy values
acc_var = tf.Variable(0, dtype=tf.float32) # variable that holds accuracy
acc_summ = tf.summary.scalar('Accuracy', acc_var) # summary to write to TensorBoard

sess = tf.Session()
for step, acc in enumerate(accuracy):
    sess.run(acc_var.assign(acc)) # update accuracy variable
    
    writer.add_summary(sess.run(acc_summ), step) # add summary

writer.flush() # make sure everything is written to disk
writer.close() # not really needed, but good habit

In [ ]:
# Visualize a basic graph
import tensorflow as tf
tf.reset_default_graph()

a = tf.Variable(2, name='a')
b = tf.Variable(3, name='b')
c = tf.add(a, b, name='Add')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter('./graphs', graph=sess.graph)

print(sess.run(c)) # -> 5

writer.flush()
writer.close()

In [ ]:
# Visualize a giant graph
import tensorflow as tf
tf.reset_default_graph()

a = tf.Variable(2.0, name='a')
b = tf.Variable(3.0, name='b')
c = tf.Variable(7.0, name='c')

d = tf.multiply(a, b, name='d-mul')
e = tf.add(b, c, name='e-add')
f = tf.subtract(e, a, name='f-sub')
g = tf.multiply(d, b, name='g-mul')
h = tf.divide(g, d, name='h-div')
i = tf.add(h, f, name='i-add')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter('./graphs', graph=sess.graph)

print(sess.run(i)) # -> 11

writer.flush()
writer.close()

In [ ]:
# Visualize a U-Net generator
import tensorflow as tf
tf.reset_default_graph()

from keras.models import Sequential, Model
from keras.layers import Activation, Concatenate, Conv2D, Conv2DTranspose, Input, LeakyReLU
from keras.initializers import RandomNormal
from keras_contrib.layers import InstanceNormalization # pip install git+https://www.github.com/keras-team/keras-contrib.git

img_rows, img_cols, channels = 256, 256, 3
weight_initializer = RandomNormal(stddev=0.02)

# "dk denotes a 3×3 Convolution-InstanceNorm-ReLU with k filters and stride 2"
def dk(k, use_instancenorm=True):
    block = Sequential()
    block.add(Conv2D(k, (3, 3), strides=2, padding='same', kernel_initializer=weight_initializer))
    if use_instancenorm:
        block.add(InstanceNormalization(axis=-1))
    block.add(Activation('relu'))

    return block

# "uk denotes a 3×3 fractional-strided-ConvolutionInstanceNorm-ReLU layer with k filters and stride ½"
def uk(k):
    block = Sequential()
    block.add(Conv2DTranspose(k, (3, 3), strides=2, padding='same', kernel_initializer=weight_initializer))
    block.add(InstanceNormalization(axis=-1))
    block.add(Activation('relu'))

    return block

def generator():
    gen_input = Input(shape=(img_rows, img_cols, channels))
    
    # Layers for the encoder part of the model
    encoder_layers = [
        dk(64, False),
        dk(128),
        dk(256),
        dk(512),
        dk(512),
        dk(512),
        dk(512),
        dk(512)
    ]

    # Layers for the decoder part of the model
    decoder_layers = [
        uk(512),
        uk(512),
        uk(512),
        uk(512),
        uk(256),
        uk(128),
        uk(64)
    ]

    gen = gen_input

    # Add all the encoder layers, and keep track of them for skip connections
    skips = []
    for layer in encoder_layers:
        gen = layer(gen)
        skips.append(gen)
    
    skips = skips[::-1][1:] # Reverse for looping and get rid of the layer that directly connects to decoder

    # Add all the decoder layers and skip connections
    for skip_layer, layer in zip(skips, decoder_layers):
        gen = layer(gen)
        gen = Concatenate()([gen, skip_layer])

    
    # Final layer
    gen = Conv2DTranspose(channels, (3, 3), strides=2, padding='same', kernel_initializer=weight_initializer, activation='tanh')(gen)
    
    # Compose model
    return Model(gen_input, gen)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

model = generator()
model.compile(optimizer='adam', loss='mse')

writer = tf.summary.FileWriter('./graphs', graph=sess.graph)

writer.flush()
writer.close()

In [ ]:
# Make a simple normal distribution
import tensorflow as tf

normal_dist = tf.random_normal(shape=(10, 10, 10, 10, 10), mean=0, stddev=1) # tensor to be logged, shape is irrelevant
summary = tf.summary.histogram("NormalDistribution", normal_dist) # summary that logs tensor

sess = tf.Session()
writer = tf.summary.FileWriter('./graphs')

writer.add_summary(sess.run(summary))

writer.flush()
writer.close()

In [ ]:
# Stack 100 normal distributions
import tensorflow as tf

mean = tf.placeholder(tf.float32)

normal_dist = tf.random_normal(shape=(10, 10, 10, 10, 10), mean=mean, stddev=1) # tensor to be logged, shape is irrelevant
summary = tf.summary.histogram("NormalDistribution", normal_dist) # summary that logs tensor

sess = tf.Session()
writer = tf.summary.FileWriter('./graphs')

for i in range(100):
    writer.add_summary(sess.run(summary, feed_dict={mean: i/50}), global_step=0)

writer.flush()
writer.close()

In [ ]:
# Visualize a multimodal distributon
import tensorflow as tf

mean = tf.placeholder(tf.float32)

normal_dist_0 = tf.random_normal(shape=(10000,), mean=-mean, stddev=0.5)
normal_dist_1 = tf.random_normal(shape=(10000,), mean=mean, stddev=0.5)

normal_dist_cat = tf.concat([normal_dist_0, normal_dist_1], axis=0) # concatenate both normal dists
summary = tf.summary.histogram("MultimodalDistribution", normal_dist_cat) # summary that logs tensor

sess = tf.Session()
writer = tf.summary.FileWriter('./graphs')

for i in range(100):
    writer.add_summary(sess.run(summary, feed_dict={mean: i/20}), global_step=0)

writer.flush()
writer.close()

In [ ]:
# Visualize some random nosie
import tensorflow as tf

image = tf.random_uniform((1, 210, 160, 3), 0, 256, dtype=tf.float32) # batch_size, height, width, channels
img_summary = tf.summary.image("InputImage", image)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

writer = tf.summary.FileWriter('./graphs')
x = sess.run(img_summary)
writer.add_summary(x)

writer.flush()
writer.close()

In [ ]:
# Visuaize the first 4 examples of MNIST
import tensorflow as tf
import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

images = x_train[:4].reshape((-1, 28, 28, 1))
img_summary = tf.summary.image("TrainingExamples", images, max_outputs=4)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

writer = tf.summary.FileWriter('./graphs')
x = sess.run(img_summary)
writer.add_summary(x)

writer.flush()
writer.close()

In [ ]:
# Display some text
import tensorflow as tf

text = tf.convert_to_tensor('Hello world!')
txt_summary = tf.summary.text("Text", text)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

writer = tf.summary.FileWriter('./graphs')
x = sess.run(txt_summary)
writer.add_summary(x)

writer.flush()
writer.close()

In [ ]:
# Merge summaries
import tensorflow as tf

# Image summary
image = tf.random_uniform((1, 210, 160, 3), 0, 256, dtype=tf.float32) # batch_size, height, width, channels
img_summary = tf.summary.image("InputImage", image, max_outputs=10)

# Histogram summary
normal_dist = tf.random_normal(shape=(10, 10, 10, 10, 10), mean=0, stddev=1) # tensor to be logged, shape is irrelevant
hist_summary = tf.summary.histogram("NormalDistribution", normal_dist) # summary that logs tensor

# Merge summaries
merged_summ = tf.summary.merge([img_summary, hist_summary])

# Start session
sess = tf.Session()
sess.run(tf.initialize_all_variables())

# Init. writer and add summary
writer = tf.summary.FileWriter('./graphs')
writer.add_summary(sess.run(merged_summ))

writer.flush()
writer.close()

In [ ]:
# JUPYTER-NOTEBOOK NOTE: If you have not restarted your Notebook before running this, this will report ALL
# summaries from previous cells included

# Merge summaries
import tensorflow as tf

# Image summary
image = tf.random_uniform((1, 210, 160, 3), 0, 256, dtype=tf.float32) # batch_size, height, width, channels
img_summary = tf.summary.image("InputImage", image, max_outputs=10)

# Histogram summary
normal_dist = tf.random_normal(shape=(10, 10, 10, 10, 10), mean=0, stddev=1) # tensor to be logged, shape is irrelevant
hist_summary = tf.summary.histogram("NormalDistribution", normal_dist) # summary that logs tensor

# Merge summaries
merged_summ = tf.summary.merge_all() # no need to specify which summaries to merge

# Start session
sess = tf.Session()
sess.run(tf.initialize_all_variables())

# Init. writer and add summary
writer = tf.summary.FileWriter('./graphs')
writer.add_summary(sess.run(merged_summ))

writer.flush()
writer.close()

In [ ]:
# Use the TensorBoard callback in a Keras model
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128), # we will visualize neurons in this layer
    Dropout(0.5),
    Dense(10, activation='softmax')
])

tensorboard_callback = keras.callbacks.TensorBoard('./graphs',
    histogram_freq=1, # how often to compute activation and weight histograms
    write_graph=True, # visualize the graph
    write_grads=True, # visual gradient histogram
    write_images=True, # visualize model weights as an image
    # embeddings_freq=1, # how often to visualize embeddings
    # embeddings_layer_names=['...'], # names of embedding layers to visualize; wouldn't work on this model
    update_freq='epoch' # update TensorBoard every epoch
)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])